In [9]:
#pip install selenium
#pip install webdriver-manager
#pip install BeautifulSoup4
import pandas as pd
import numpy as np
import re
import requests
import webbrowser
import seaborn as sns
import urllib.request, json 
from IPython.core.display import display, HTML
from itertools import chain
from datetime import date
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium import webdriver
from datetime import datetime, date, timedelta
pd.set_option("display.max.columns", None)
pd.set_option("display.max.rows", None)
#driver = webdriver.Chrome("C:/Users/Valentin/Documents/Winamax/chromedriver")
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST driver version for 96.0.4664
Driver [C:\Users\Valentin\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [10]:
#Function Soup
def get_page(special_match):
    driver.get(special_match)
    try:
        element = driver.find_element_by_class_name('banner-actions-container')
        element.click()
    except:
        do = "Nothing"
            
get_page("http://www.espn.com/nba/players")
soup = BeautifulSoup(driver.page_source)
table_data = []

for teams in soup.find(class_='span-4').findAll(class_='mod-container mod-open-list mod-no-footer mod-teams-list-small'):
    for href in teams.find_all('a', href=re.compile("http")):
        team = href["href"].replace("team/_/name/","team/roster/_/name/")
        get_page(team)
        soup = BeautifulSoup(driver.page_source)
        for players in soup.find(class_='Table__TBODY').findAll('tr'):
            row_data = []
            for cell in players.findAll('td'):
                row_data.append(cell.text)
            data_item = {"URL": players.find('a', href=re.compile("nba/player/_/id"))["href"],
                         "Name": row_data[1],
                         "FullName": players.find('a', href=re.compile("nba/player/_/id"))["href"].replace("https://www.espn.com/nba/player/_/id/","").split("/")[1].replace("-", "").replace("'", "").replace(".", "").replace(" ", "").lower(),
                         "Team": soup.find(class_="ClubhouseHeader__Name ttu flex items-start n2").text,
                         "POS": row_data[2],
                         "AGE": row_data[3],
                         "HT": row_data[4],
                         "WT": row_data[5],
                         "COLLEGE": row_data[6],
                         "SALARY": row_data[7]                 
                        }
            table_data.append(data_item)
pd.DataFrame(table_data).to_csv('Players_NBA.csv', index=False, encoding='utf-8')

AttributeError: 'NoneType' object has no attribute 'findAll'

In [8]:
#Function Soup
def get_page(special_match):
    driver.get(special_match)
    try:
        element = driver.find_element_by_class_name('banner-actions-container')
        element.click()
    except:
        do = "Nothing"
            
get_page("http://www.espn.com/nba/players")
soup = BeautifulSoup(driver.page_source)
table_data = []
i = 0
for teams in soup.find(class_='span-4').findAll(class_='mod-container mod-open-list mod-no-footer mod-teams-list-small'):
    for href in teams.find_all('a', href=re.compile("http")):
        team = href["href"].replace("team/_/name/","team/stats/_/name/")
        t = team.replace("http://www.espn.com/nba/team/stats/_/name/","").split("/")[0]
        print(t)
        
        while team[-1] != '/':
            team = team[:-1]
        i=i+1
        
        for home in ["split/33", "split/34"]:
            #split/33 = Home
            #split/34 = Away
            get_page(team + home)
            soup = BeautifulSoup(driver.page_source)
            i = 0
            for tables in soup.find(class_='Wrapper Card__Content').find(class_='ResponsiveTable ResponsiveTable--fixed-left mt5 remove_capitalize'):
                for row in tables.findAll('tr'):
                    row_data = []
                    for cell in row.findAll('td'):
                        row_data.append(cell.text)
                    i = i+1
                if(i>0):
                    if(len(row_data)>1):
                        data_item = {
                                     "Place": "Home" if  home == "split/33" else "Away",
                                     "Team": soup.find(class_="ClubhouseHeader__Name ttu flex items-start n2").text,
                                     "id": t,
                                     "GP": row_data[0],
                                     "GS": row_data[1],
                                     "MIN": row_data[2],
                                     "PTS": row_data[3],
                                     "OR": row_data[4],
                                     "DR": row_data[5],
                                     "REB": row_data[6],
                                     "AST": row_data[7],
                                     "STL": row_data[8],
                                     "BLK": row_data[9],
                                     "TO": row_data[10],
                                     "PF": row_data[11],
                                     "AST/TO": row_data[12],
                                     "PER": row_data[13]
                                    }
                        table_data.append(data_item)
pd.DataFrame(table_data).to_csv('Teams_NBA.csv', index=False, encoding='utf-8')

bos
bkn
ny
phi
tor
gs
lac
lal
phx
sac
chi
cle
det
ind
mil
dal
hou
mem
no
sa
atl
cha
mia
orl
wsh
den
min
okc
por
utah


In [228]:
table_HOMEAWAY = []
for home in ["/split/33", "/split/34"]:
    get_page("https://www.espn.com/nba/team/stats/_/name/"+ "lal" + home)
    soup = BeautifulSoup(driver.page_source)
    i = 0
    for tables in soup.find(class_='Wrapper Card__Content').find(class_='ResponsiveTable ResponsiveTable--fixed-left mt5 remove_capitalize'):
        for row in tables.findAll('tr'):
            row_data = []
            for cell in row.findAll('td'):
                row_data.append(cell.text)
            i = i+1
        if(i>0):
            if(len(row_data)>1):
                data_item = {
                             "Place": "Home" if  home == "/split/33" else "Away",
                             "Team": soup.find(class_="ClubhouseHeader__Name ttu flex items-start n2").text,
                             "GP": row_data[0],
                             "GS": row_data[1],
                             "MIN": row_data[2],
                             "PTS": row_data[3],
                             "OR": row_data[4],
                             "DR": row_data[5],
                             "REB": row_data[6],
                             "AST": row_data[7],
                             "STL": row_data[8],
                             "BLK": row_data[9],
                             "TO": row_data[10],
                             "PF": row_data[11],
                             "AST/TO": row_data[12],
                             "PER": row_data[13]
                            }
                table_HOMEAWAY.append(data_item)
pd.DataFrame(table_HOMEAWAY)

,Place,Team,GP,GS,MIN,PTS,OR,DR,REB,AST,STL,BLK,TO,PF,AST/TO,PER
0,Home,Los AngelesLakers,11,,,110.4,10.1,34.9,45.0,24.5,9.1,6.4,16.5,20.5,1.5,
1,Away,Los AngelesLakers,3,,,110.0,10.0,41.7,51.7,23.0,8.0,5.7,16.3,20.0,1.4,
